In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import time

## Parameters

In [ ]:
keep_len=12 
sensor_num=6

## Load data

In [ ]:
train_adjmatrix = np.load('train_adjmatrix.npy')
test_adjmatrix = np.load('test_adjmatrix.npy')

train_inputdata = np.load('train_inputdata.npy')
test_inputdata = np.load('test_inputdata.npy')

train_pollutiondata = np.load('train_pollutiondata.npy')
test_pollutiondata = np.load('test_pollutiondata.npy')

print(train_adjmatrix.shape)
print(test_adjmatrix.shape)

print(train_inputdata.shape)
print(test_inputdata.shape)

print(train_pollutiondata.shape)
print(test_pollutiondata.shape)



In [ ]:
train_num = train_adjmatrix.shape[0]
test_num = test_adjmatrix.shape[0]

train_dataset= np.concatenate((train_adjmatrix.reshape(train_num,-1),train_inputdata.reshape(train_num,-1), train_pollutiondata),1)
test_dataset=  np.concatenate((test_adjmatrix.reshape(test_num,-1),test_inputdata.reshape(test_num,-1), test_pollutiondata),1)

print(train_dataset.shape)
print(test_dataset.shape)


# Deconder Topology Model
# DNN_CNN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class fc_cnn_model(nn.Module):
    def __init__(self, nnode, nfeat, enconder_out_dim):
        super(fc_cnn_model, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(1, 3), padding=(0, 1))
        self.cnn2 = nn.Conv2d(in_channels=4, out_channels=16, kernel_size=(1, 3), padding=(0, 1))
        self.cnn3 = nn.Conv2d(in_channels=16, out_channels=1, kernel_size=(1, 1))
        
        self.fc_encoder = nn.Linear(nnode*nnode,enconder_out_dim)
        
        self.fc = nn.Linear(nnode*nfeat+enconder_out_dim, nnode)
        
        
        self.nnode = nnode
        self.nfeat = nfeat
        self.enconder_out_dim = enconder_out_dim
        


    def forward(self, x, adj):
        
        x = x.reshape(-1, 1, self.nnode, self.nfeat)

        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn2(x))
        x = F.relu(self.cnn3(x))

        x = x.reshape(-1, self.nnode* self.nfeat)
        
        adj = adj.reshape(-1, self.nnode**2)
        adj_coder = F.relu(self.fc_encoder(adj))

        x =torch.cat((x, adj_coder),1)
        
        x = self.fc(x)
        x = x.reshape(-1,self.nnode)

        return x

In [ ]:
from torch.utils.data import DataLoader
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)


In [ ]:
# Model and optimizer

model = fc_cnn_model(nnode=100,
            nfeat=keep_len,
            enconder_out_dim=100)

optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)

criterion = nn.CrossEntropyLoss()

model.cuda()

lr_milestones = [30, 50]
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=lr_milestones, gamma=0.5)



In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
    

In [ ]:
import time
import math

if not os.path.isdir('checkpoint'):
    os.mkdir('checkpoint')
    
if not os.path.isdir('results'):
    os.mkdir('results')
    
best_loss = float('inf')
# Train model
for epoch in range(60):
    model.train()
    scheduler.step()
    total_loss_train = AverageMeter()
    for index, data in enumerate(trainloader):

        model.train()
        batch_size =len(data)

        train_adj = data[:,:10000].float()
        train_input_x = data[:,10000:-1].float()
        train_Y = data[:,-1].long()

        train_adj = train_adj.reshape(batch_size, 100, 100)
        
        train_input_x = train_input_x.reshape(batch_size, 100, keep_len)

        train_adj, train_input_x, train_Y  = train_adj.cuda(), train_input_x.cuda(), train_Y.cuda()

        train_Y_hat = model(train_input_x, train_adj)
        

        batch_loss = criterion(train_Y_hat, train_Y)

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        total_loss_train.update(batch_loss.item(), batch_size)
        if index%10==0:
            print('Epoch: {}, index: {}, train loss: {:.4f}'.format(epoch+1,int(index), total_loss_train.avg))
            file = open("./results/loss2.txt","a")
            file.write("Epoch = {}, index = {}  ".format(epoch+1,int(index)))
            file.write("\n")
            file.write("train_loss = {:.4f}  ".format(total_loss_train.avg))
            file.write("\n")
            file.close

    model.eval()
    total_loss_test = AverageMeter()
    right_num_top1 = 0
    right_num_top5 = 0
    right_num_top10 = 0
    for index_test, test_data in enumerate(testloader):
        
        batch_size =len(test_data)

        test_adj = test_data[:,:10000].float()
        test_input_x = test_data[:,10000:-1].float()
        test_Y = test_data[:,-1].long()

        test_adj = test_adj.reshape(batch_size, 100, 100)
        
        test_input_x = test_input_x.reshape(batch_size, 100, keep_len)

        test_adj, test_input_x, test_Y  = test_adj.cuda(), test_input_x.cuda(), test_Y.cuda()

        test_Y_hat = model(test_input_x, test_adj)


        loss_test = criterion(test_Y_hat, test_Y)

        total_loss_test.update(loss_test.item(), test_input_x.size(0))
        
        gt_node =  test_Y.detach().cpu().numpy()
        
        test_Y_hat = F.log_softmax(test_Y_hat).detach().cpu().numpy()
        gnn_pre_node = np.argmax(test_Y_hat, axis=1)
        
        right_num_top1+=np.sum(gt_node==gnn_pre_node)
        for j in range(batch_size):
            gt_node_sample = gt_node[j]
            gnn_pre_node_top5 = np.argpartition(test_Y_hat[j,:], -5)[-5:]
            gnn_pre_node_top10 = np.argpartition(test_Y_hat[j,:], -10)[-10:]
            if np.isin(gt_node_sample, gnn_pre_node_top5):
                right_num_top5+=1
            if np.isin(gt_node_sample, gnn_pre_node_top10):
                right_num_top10+=1
                
    print('*'*50)
    print('Epoch = ', epoch+1)        
    print('test loss: {:.4f}, test accuracy top1:{:.5f}, test accuracy top5:{:.5f}, test accuracy top10:{:.5f}'.format(total_loss_test.avg, right_num_top1/len(test_dataset), right_num_top5/len(test_dataset), right_num_top10/len(test_dataset)))
    
    file = open("./results/results2.txt","a")
    file.write("test_loss = {:.4f}  ".format(total_loss_test.avg))
    file.write("test_accuracy_top1 = {:.5f}  ".format(right_num_top1/len(test_dataset)))
    file.write("test_accuracy_top5 = {:.5f}  ".format(right_num_top5/len(test_dataset)))
    file.write("test_accuracy_top10 = {:.5f}  ".format(right_num_top10/len(test_dataset)))
    file.write("\n")
    file.close
    
    if total_loss_test.avg<best_loss:
        best_loss = total_loss_test.avg
        model_state = {
            'net_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch':epoch,
            'best_loss':best_loss
        }
       
        save_point = './checkpoint/2_DNN_CNN'
        torch.save(model_state, save_point)

